In [16]:
import glob
from pathlib import Path
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import dateparser
import ast
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime
outdoors_places = ['Olympiastadion', 'Tierpark Hellabrunn','Olympiapark']
indoors_places = ['Bayerisches Nationalmuseum','Neue Pinakothek',
                   'Bayerisches Staatsorchester','Muenchner Stadtmuseum',
                   'Museum Brandhorst','Nationaltheater','Residenztheater',
                   'Alte Pinakothek','Museum Mensch und Natur','Muenchner Philharmoniker',
                   'Prinzregententheater','Pinakothek der Moderne','Staedtische Galerie im Lenbachhaus',
                   'Deutsches Museum','Theater am Gaertnerplatz']
train_user_entries = [{'userId':'01','origin': 'Berlin', 'accomodation': 'Maxvorstadt', 'visit_type': 'alone', 'place_pref': 'indoors'},
                {'userId':'02','origin': 'Prag', 'accomodation': 'Hadern', 'visit_type': 'with_family', 'place_pref': 'outdoors'},
                {'userId':'03','origin': 'Berlin', 'accomodation': 'Hadern', 'visit_type': 'alone', 'place_pref': 'indoors'},
                {'userId':'04','origin': 'Prag', 'accomodation': 'Maxvorstadt', 'visit_type': 'alone', 'place_pref': 'outdoors'}]
test_user_entries = [{'userId':'01','origin': 'Bonn', 'accomodation': 'Maxvorstadt', 'visit_type': 'with_family', 'place_pref': 'outdoors'},
                {'userId':'02','origin': 'Prag', 'accomodation': 'Maxvorstadt', 'visit_type': 'alone', 'place_pref': 'outdoors'},
                {'userId':'03','origin': 'Bonn', 'accomodation': 'Hadern', 'visit_type': 'alone', 'place_pref': 'indoors'},
                {'userId':'04','origin': 'Paris', 'accomodation': 'Maxvorstadt', 'visit_type': 'alone', 'place_pref': 'indoors'}]
train_df_entries = pd.DataFrame(train_user_entries)
test_df_entries = pd.DataFrame(test_user_entries)

In [17]:
train_df_entries

,userId,origin,accomodation,visit_type,place_pref
0,01,Berlin,Maxvorstadt,alone,indoors
1,02,Prag,Hadern,with_family,outdoors
2,03,Berlin,Hadern,alone,indoors
3,04,Prag,Maxvorstadt,alone,outdoors


In [18]:
test_df_entries

,userId,origin,accomodation,visit_type,place_pref
0,01,Bonn,Maxvorstadt,with_family,outdoors
1,02,Prag,Maxvorstadt,alone,outdoors
2,03,Bonn,Hadern,alone,indoors
3,04,Paris,Maxvorstadt,alone,indoors


In [4]:
placesFeatures = pd.read_csv('../data/places_features.csv')
placesFeatures

,Unnamed: 0,place,rating,#_of_visits,city_district_metric,all_metric_score,city_district,type_door
0,0,Alte Pinakothek,46.023297,252,0.122408,0.061204,Maxvorstadt,indoors
1,1,Bayerisches Nationalmuseum,43.010965,151,0.034811,0.017405,Altstadt-Lehel,indoors
2,2,Deutsches Museum,40.666352,155,0.122542,0.061271,Ludwigsvorstadt-Isarvorstadt,indoors
3,3,Muenchner Stadtmuseum,38.904321,104,0.034811,0.017405,Altstadt-Lehel,indoors
4,4,Museum Brandhorst,43.219728,141,0.122408,0.061204,Maxvorstadt,indoors
5,5,Museum Mensch und Natur,45.809524,47,0.060613,0.030306,Neuhausen-Nymphenburg,indoors
6,6,Nationaltheater,45.821813,54,0.034811,0.017405,Altstadt-Lehel,indoors
7,7,Neue Pinakothek,43.377083,153,0.122408,0.061204,Maxvorstadt,indoors
8,8,Olympiapark,45.517419,155,0.036830,0.018415,Milbertshofen-Am Hart,outdoors
9,9,Olympiastadion,40.768044,155,0.036830,0.048903,Milbertshofen-Am Hart,outdoors


In [ ]:
# Install your licensed copy of GraphLab Create
!pip install --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.1/ferouk.ferjani@tum.de/0EE2-8E9E-216D-D7E2-FD86-39D1-B598-004A/GraphLab-Create-License.tar.gz

In [ ]:
import graphlab


# Convert the pandas dataframes to graph lab SFrames
train_data = graphlab.SFrame(train_df_entries)
test_data = graphlab.SFrame(test_df_entries)

# Train the model
collab_filter_model = graphlab.item_similarity_recommender.create(train_data, 
                                                                  user_id='user_id', 
                                                                  item_id='accomodation', 
                                                                  target='rating', 
                                                                  similarity_type='cosine')
                                                                  
# Make recommendations
which_user_ids = [1, 2, 3]
how_many_recommendations = 5
item_recomendation = collab_filter_model.recommend(users=which_user_ids,
                                                   k=how_many_recommendations)

In [ ]:
import scipy.sparse
from scipy.spatial.distance import correlation


def favoritePlace(activeUser,N):
    topPlace=pd.DataFrame.sort_values(
        data[data.userId==activeUser],['rating'],ascending=[0])[:N]
    return list(topPlace.title)

userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['itemId'])


def similarity(user1,user2):
    try:
        user1=np.array(user1)-np.nanmean(user1)
        user2=np.array(user2)-np.nanmean(user2)
        commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
        if len(commonItemIds)==0:
           return 0
        else:
           user1=np.array([user1[i] for i in commonItemIds])
           user2=np.array([user2[i] for i in commonItemIds])
           return correlation(user1,user2)
    except ZeroDivisionError:
        print("You can't divide by zero!")



def nearestNeighbourRatings(activeUser,K):
    try:
        similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['Similarity'])
        for i in userItemRatingMatrix.index:
            similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],userItemRatingMatrix.loc[i])
        similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,['Similarity'],ascending=[0])
        nearestNeighbours=similarityMatrix[:K]
        neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
        predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
        for i in userItemRatingMatrix.columns:
            predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
            for j in neighbourItemRatings.index:
                if userItemRatingMatrix.loc[j,i]>0:
                   predictedRating += (userItemRatingMatrix.loc[j,i]-np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
                predictItemRating.loc[i,'Rating']=predictedRating
    except ZeroDivisionError:
        print("You can't divide by zero!")            
    return predictItemRating


def topNRecommendations(activeUser,N):
    try:
        predictItemRating=nearestNeighbourRatings(activeUser,10)
        placeAlreadyWatched=list(userItemRatingMatrix.loc[activeUser]
                              .loc[userItemRatingMatrix.loc[activeUser]>0].index)
        predictItemRating=predictItemRating.drop(placeAlreadyWatched)
        topRecommendations=pd.DataFrame.sort_values(predictItemRating,
                                                ['Rating'],ascending=[0])[:N]
        topRecommendationTitles=(placeInfo.loc[placeInfo.itemId.isin(topRecommendations.index)])
    except ZeroDivisionError:
        print("You can't divide by zero!")
    return list(topRecommendationTitles.title)


# activeUser=int(input("Enter userid: "))
# #print("The user's favorite places are: ")
# #print(favoritePlace(activeUser,5))
# print("The recommended places for you are: ")
# print(topNRecommendations(activeUser,4))

# I dont know if we really need a ML model for achieveing this puporse with the actual datasets. Either we merge them both in one dataset with dropping the metrics because they are mainly there for the prediction or we could rely on a scoring system (as Oumeima did) and recommend the most suitable places just through some "if" statements